In [1]:
%%capture
import re
import matplotlib.pyplot as plt
import pandas as pd
import nltk
nltk.download('stopwords')

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))


import tensorflow as tf
import numpy as np
from wordcloud import WordCloud
from keras.utils import to_categorical

# Classification Model
!pip install transformers
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Model Training
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
%%capture
#Get Original dataset
!rm -rf /content/Fake-News-Detection-Text-Classification
!git clone 'https://github.com/shariqfz/Fake-News-Detection-Text-Classification.git'
# !tar -zxvf '/content/Fake-News-Detection-Text-Classification/data/FakeNewsData.tar.gz'

In [ ]:
# hate_speech_path = "/content/Fake-News-Detection-Text-Classification/data/HateSpeech/hate_speech_data.csv"

# data = pd.read_csv(hate_speech_path)
# data.head()

In [3]:
combined_data = pd.read_csv("/content/Fake-News-Detection-Text-Classification/data/HateSpeech/hate_speech_cleaned_and_stopword_removed_combined_data.csv")
combined_data = combined_data.dropna()
combined_data.head(10)

,Unnamed: 0,tweet,class
0,0,niggas always yelling fuck bitches get money m...,1
1,1,heard looking candy man bitch,1
2,2,awww meat meat feelinsi ainn n feelings bitch,1
3,3,well gahdamn rt wooooow good job mommy cutest ...,0
4,4,rt black guy school asked colored printers lib...,0
5,5,whizzing crackers cats got amp playing hockey ...,2
6,6,rt exactly shit saidgtgtgt nobody think mimi o...,0
7,7,chillin homie whats left british rapper turned...,2
8,8,yall tl need sleep yall got school tomorrow bi...,1
9,9,aint never seen bitch obsessed nigga im obsess...,1


In [4]:
combined_data = combined_data.dropna()

## Bert Hate Speech detector

In [5]:
X = combined_data.sample(frac=1)    # shuffle the data

Y = X['class'].to_numpy().astype('float32').reshape(-1,1)
Y = to_categorical(Y, num_classes=3)    # Convert to one-hot encoding
X = X.tweet.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
    X, Y,
    train_size=0.9,
    test_size=0.1,
    stratify=Y,
    random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train,
    train_size=0.9,
    test_size=0.1,
    stratify=y_train,
    random_state=42
)

In [6]:
# Start of Code taken from https://www.kaggle.com/code/utkarshsaxenadn/fake-vs-real-news-detection-bert-acc-100/notebook#BERT-Classification-Model

# Model Callbacks
LR = 5e-5
model_name = "BERTHateSpeechDetector_" + str(LR)
MODEL_CALLBACKS = [ModelCheckpoint(model_name, save_best_only=True)]

bert_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(
    bert_name,
    padding = "max_length",
    do_lower_case = True,
    add_special_tokens = True,
)

X_train_encoded = tokenizer(
    X_train.tolist(),
    padding = True,
    truncation = True,
    return_tensors = "tf"
).input_ids

X_valid_encoded = tokenizer(
    X_valid.tolist(),
    padding = True,
    truncation = True,
    return_tensors = "tf"
).input_ids

X_test_encoded = tokenizer(
    X_test.tolist(),
    padding = True,
    truncation = True,
    return_tensors = "tf"
).input_ids

# train_ds = tf.data.Dataset.from_tensor_slices((X_train_encoded, y_train)).shuffle(len(X_train)).batch(8).prefetch(tf.data.AUTOTUNE)
# valid_ds = tf.data.Dataset.from_tensor_slices((X_valid_encoded, y_valid)).shuffle(len(X_valid)).batch(8).prefetch(tf.data.AUTOTUNE)
# test_ds  = tf.data.Dataset.from_tensor_slices((X_test_encoded, y_test)).shuffle(len(X_test)).batch(8).prefetch(tf.data.AUTOTUNE)

train_ds = tf.data.Dataset.from_tensor_slices((X_train_encoded, y_train))
train_ds = train_ds.shuffle(len(X_train))
train_ds = train_ds.batch(8)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

valid_ds = tf.data.Dataset.from_tensor_slices((X_valid_encoded, y_valid))
valid_ds = valid_ds.shuffle(len(X_valid))
valid_ds = valid_ds.batch(8)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

test_ds  = tf.data.Dataset.from_tensor_slices((X_test_encoded, y_test))
test_ds = test_ds.shuffle(len(X_test))
test_ds = test_ds.batch(8)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)


bert_model = TFAutoModelForSequenceClassification.from_pretrained(bert_name, num_labels = 3)

bert_model.compile(
    optimizer = Adam(learning_rate = LR),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = [
        tf.keras.metrics.BinaryAccuracy(name="Accuracy"),
        tf.keras.metrics.Precision(name="Precision"),
        tf.keras.metrics.Recall(name="Recall"),
        # tf.keras.metrics.F1Score(name='F1_Score'),
        # tfa.metrics.F1Score(num_classes=2, threshold=0.5, name='F1_Score'),
        tf.keras.metrics.AUC(name='AUC')
    ]
)

model_history = bert_model.fit(
    train_ds,
    validation_data = valid_ds,
    epochs = 4,
    batch_size = 16,
    callbacks = MODEL_CALLBACKS
)

model_history = pd.DataFrame(model_history.history)

# Save the mdoel
# bert_model.save(model_name)

# End of Code taken from https://www.kaggle.com/code/utkarshsaxenadn/fake-vs-real-news-detection-bert-acc-100/notebook#BERT-Classification-Model

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1/4
426/426 [==============================] - 179s 238ms/step - loss: 0.7882 - Accuracy: 0.7672 - Precision: 0.6920 - Recall: 0.5437 - AUC: 0.7614 - val_loss: 0.6091 - val_Accuracy: 0.8351 - val_Precision: 0.7290 - val_Recall: 0.8042 - val_AUC: 0.8648
Epoch 2/4
426/426 [==============================] - 93s 218ms/step - loss: 0.5490 - Accuracy: 0.8570 - Precision: 0.7534 - Recall: 0.8489 - AUC: 0.8804 - val_loss: 0.5901 - val_Accuracy: 0.8025 - val_Precision: 0.6598 - val_Recall: 0.8413 - val_AUC: 0.8565
Epoch 3/4
426/426 [==============================] - 92s 217ms/step - loss: 0.5004 - Accuracy: 0.8556 - Precision: 0.7387 - Recall: 0.8771 - AUC: 0.8875 - val_loss: 0.5167 - val_Accuracy: 0.8386 - val_Precision: 0.7171 - val_Recall: 0.8519 - val_AUC: 0.8740
Epoch 4/4
426/426 [==============================] - 101s 237ms/step - loss: 0.4169 - Accuracy: 0.8710 - Precision: 0.7582 - Recall: 0.9000 - AUC: 0.9065 - val_loss: 0.5004 - val_Accuracy: 0.8616 - val_Precision: 0.7461 - val

In [7]:
test_loss, test_acc, test_precision, test_recall, test_auc = bert_model.evaluate(test_ds, verbose = 0)

print(f"Test Loss      : {test_loss}")
print(f"Test Accuracy  : {test_acc}")
print(f"Test Precision : {test_precision}")
print(f"Test Recall    : {test_recall}")
# print(f"Test F1        : {tests_f1}")
print(f"Test AUC       : {test_auc}")

Test Loss      : 0.41877222061157227
Test Accuracy  : 0.8809523582458496
Test Precision : 0.7743902206420898
Test Recall    : 0.9071428775787354
Test AUC       : 0.90227872133255


## Prediction on Fake News data

In [ ]:
fake_news_data = pd.read_csv('/content/Fake-News-Detection-Text-Classification/preprocessed_data/stemmed_and_stopword_removed_combined_set.csv')

fake_news_data = fake_news_data.dropna()
fake_news_data['hate_speech_class'] = None

# fake_news_data = fake_news_data.iloc[:100, :]
fake_news_data = fake_news_data.sample(1000)

# fake_news_data = fake_news_data.rename(columns={'hate_speec_class':'hate_speech_class'})
# fake_news_data.text[0]

In [ ]:
len(fake_news_data)
fake_news_data.text.iloc[900]

'kill list republican found bodi jame hodgkinsonth dem hatespeech work derang shooter tri kill republican congressmen practic basebal field alexandria virginia wa carri kill list republican includ alabama rep mo brook democrat convinc vitriol speech motiv peopl violenc thi yet anoth exampl target conserv republican lawmak go sinc trump announc hi candidacywho forget egg face trump support assault numer peopl ralli support candid trump sinc trump hate violenc ha continu inaugur berkeley antifa riotsnot democrat leader thishodgkinson spew leftw talk point facebook consist rail republican wealthi know rich pay enough tax daili caller reportsth list wa written notepad paper found shooter pocket accord multipl sourc intim knowledg situat spoke condit anonym cite sensit investig list name includ alabama rep mo brook south carolina rep jeff duncan arizona rep trent frank dc ha confirmedth fbi ha contact least one three congressmen inform inclus listnon three offic would offer comment record a

In [ ]:
def predict_text(text, model):
    tokens = tokenizer(text, return_tensors = 'tf', padding="max_length", truncation=True).input_ids
    return np.argmax(model.predict(tokens, verbose = 0).logits)

In [ ]:
count = 0
for i, row in fake_news_data.iterrows():
  count += 1
  pred = predict_text(row.text, bert_model)
  print(count, i, pred , row.text[-20:])
  fake_news_data.at[i, 'hate_speech_class'] = pred

fake_news_data.head()

In [ ]:
fake_news_data.text.iloc[900]

'kill list republican found bodi jame hodgkinsonth dem hatespeech work derang shooter tri kill republican congressmen practic basebal field alexandria virginia wa carri kill list republican includ alabama rep mo brook democrat convinc vitriol speech motiv peopl violenc thi yet anoth exampl target conserv republican lawmak go sinc trump announc hi candidacywho forget egg face trump support assault numer peopl ralli support candid trump sinc trump hate violenc ha continu inaugur berkeley antifa riotsnot democrat leader thishodgkinson spew leftw talk point facebook consist rail republican wealthi know rich pay enough tax daili caller reportsth list wa written notepad paper found shooter pocket accord multipl sourc intim knowledg situat spoke condit anonym cite sensit investig list name includ alabama rep mo brook south carolina rep jeff duncan arizona rep trent frank dc ha confirmedth fbi ha contact least one three congressmen inform inclus listnon three offic would offer comment record a

In [ ]:
fake_news_data.to_csv("hate_speech_non_concatenated.csv")

In [ ]:
def getClassMeaning(l):
  if l == 0:
    return " Hate Speech"
  elif l == 1:
    return " Offensive"
  else:
    return " Neither Hate Speech nor Offensive"

In [ ]:

data = fake_news_data.copy()
meanings = data.hate_speech_class.map(getClassMeaning)
# print(meanings)
data.text = data.text + meanings
data.text.iloc[900]

# for i, text in enumerate(sample.text):
#   newtext = getClassMeaning(sample.hate_speech_class.iloc[0])
#   oldtext = sample.text.iloc[0]
#   sample.at[i, 'newtext'] = oldtext + newtext

# sample.text.iloc[0]

data.to_csv("hate_speech_concatenated_in_text.csv")

In [ ]:
data.text.iloc[900]

'kill list republican found bodi jame hodgkinsonth dem hatespeech work derang shooter tri kill republican congressmen practic basebal field alexandria virginia wa carri kill list republican includ alabama rep mo brook democrat convinc vitriol speech motiv peopl violenc thi yet anoth exampl target conserv republican lawmak go sinc trump announc hi candidacywho forget egg face trump support assault numer peopl ralli support candid trump sinc trump hate violenc ha continu inaugur berkeley antifa riotsnot democrat leader thishodgkinson spew leftw talk point facebook consist rail republican wealthi know rich pay enough tax daili caller reportsth list wa written notepad paper found shooter pocket accord multipl sourc intim knowledg situat spoke condit anonym cite sensit investig list name includ alabama rep mo brook south carolina rep jeff duncan arizona rep trent frank dc ha confirmedth fbi ha contact least one three congressmen inform inclus listnon three offic would offer comment record a

## Predict on misclassified samples

In [ ]:
misclassifications = pd.read_csv("/content/Fake-News-Detection-Text-Classification/BERT/misclassifications_bert.csv")
CLASS_NAMES = ["Fake", "Real"]
MAPPING_DICT = {
    "Fake":0,
    "Real":1
}

In [ ]:
misclassifications_copy = misclassifications[:]
for index, sample in misclassifications_copy.iterrows():

    # _, _, text, true, prediction = sample
    text = sample['text']
    print(text)
    true = sample['true']
    model_pred = predict_text(text, model = bert_model)
    pred_text = getClassMeaning(model_pred)

    print(model_pred)

    if (True):
      # misclassifications.append({'text': text, 'true': true, 'prediction': CLASS_NAMES[int(model_pred)]})

      print(f"ORGINAL TEXT:\n\n{text[:50]}\n\nTRUE: {true}\tPREDICTED: {pred_text}\n")

      # generateWordCloud(X_test[index])
      # generateWordCloudComparative(text)
      print(f"{'-'*100}\n")
      misclassifications_copy.at[index, 'text'] = misclassifications_copy.loc[index, 'text'] + pred_text


texa politician slam tweet sent florida shoot offic texa lieuten governor dan patrick republican evangel christian sunday delet tweet mani saw offens insensit wa sent shortli deadli attack gay nightclub florida deceiv god cannot mock man reap sow read bibl vers patrick sent hi offici twitter account hour worst mass shoot us histori left least dead hi offic wa immedi avail comment ha told local media includ dalla morn new tweet schedul releas well befor shoot offic regularli send bibl vers sunday ha sinc delet replac anoth passag said delet tweet politician ha condemn samesex marriag fought lgbt right still rais ire accus patrick blame victim disgrac state thi time nation sorrow resign promin lgbt activist georg takei wrote twitter texa democrat parti call patrick apolog immedi make thi right better thi said statement patrick current countri ha yet made comment media tweet new vers sent hi account read salvat righteou come lord stronghold time troubl
0
ORGINAL TEXT:

texa politician sla

In [ ]:
misclassifications_copy.text.iloc[0]

'texa politician slam tweet sent florida shoot offic texa lieuten governor dan patrick republican evangel christian sunday delet tweet mani saw offens insensit wa sent shortli deadli attack gay nightclub florida deceiv god cannot mock man reap sow read bibl vers patrick sent hi offici twitter account hour worst mass shoot us histori left least dead hi offic wa immedi avail comment ha told local media includ dalla morn new tweet schedul releas well befor shoot offic regularli send bibl vers sunday ha sinc delet replac anoth passag said delet tweet politician ha condemn samesex marriag fought lgbt right still rais ire accus patrick blame victim disgrac state thi time nation sorrow resign promin lgbt activist georg takei wrote twitter texa democrat parti call patrick apolog immedi make thi right better thi said statement patrick current countri ha yet made comment media tweet new vers sent hi account read salvat righteou come lord stronghold time troubl Hate Speech'

In [ ]:
misclassifications_copy.to_csv("misclassifications_bert_hate_speech_appended.csv")

In [ ]:
misclassifications_copy

,Unnamed: 0,text,true,prediction
0,0,openbord liber put entir nation high alert ger...,Fake,Real
1,1,prohillari saudi princ gave american great rea...,Fake,Real
2,2,boom danish govern consid seiz migrant valuabl...,Fake,Real
3,3,san juan mayor nasti secret reveal real reason...,Fake,Real
4,4,gop board educ candid may craziest person texa...,Fake,Real
5,5,nike drop conserv boxer manni pacquiao condemn...,Fake,Real
6,6,break new violent g protest trap melania hotel...,Fake,Real
7,7,list governor told obama muslim refuge syria w...,Fake,Real
